In [19]:
import os
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from Bio import SeqIO
from Bio.PDB import *
import py3Dmol
# import MDAnalysis as mda
from vina import Vina
import requests
import xml.etree.ElementTree as ET

In [20]:
temp_drugpdb = "temp_drug.pdb"
temp_protpdb = "temp_prot.pdb"
temp_drugpdbqt = "temp_drug.pdbqt"
temp_protpdbqt = "temp_prot.pdbqt"
docking_output = "docking_output.pdbqt"

In [21]:
def prepare_ligand(smiles):
    """
    Convert SMILES to 3D structure and prepare for docking
    """
    # # Create RDKit molecule from SMILES
    # mol = Chem.MolFromSmiles(smiles)
    # mol = Chem.AddHs(mol)
    
    # # Generate 3D conformation
    # AllChem.EmbedMolecule(mol, randomSeed=42)
    # AllChem.MMFFOptimizeMolecule(mol)
    
    # # Save as PDB file
    # writer = Chem.PDBWriter(temp_drugpdb)
    # writer.write(mol)
    # writer.close()

    # os.system(f"obabel {temp_drugpdb} -O {temp_drugpdbqt} -xh -xr")

    conversion_result = os.system(f"obabel -:'{smiles}' -O {temp_drugpdbqt} -xr --gen3d --addhydrogens -p -xh -xn")
    if conversion_result != 0:
            raise RuntimeError(f"Error converting drug PDB to PDBQT. Command failed.")
    
    return temp_drugpdbqt

def prepare_protein(uniprot_id):
    """
        Check if a UniProt ID exists in the AlphaFold database and fetch the PDB file if available.
    """
    print("Checking in AlphaFold database:....")
    # Strip version suffix (e.g., ".1") if present
    clean_uniprot_id = uniprot_id.split('.')[0]

    alphafold_url = f"https://alphafold.ebi.ac.uk/files/AF-{clean_uniprot_id}-F1-model_v4.pdb"
    response = requests.get(alphafold_url)
        
    if response.status_code == 200:
        with open(temp_protpdb,"w") as f:
            f.write(response.text)
        os.system(f"obabel {temp_protpdb} -O {temp_protpdbqt} -xh -xr")
        # clean_pdb(temp_protpdb)
        return temp_protpdbqt
    else:
        """Fetch PDB IDs associated with a UniProt ID using the UniProt API."""
        url = f"https://www.uniprot.org/uniprot/{uniprot_id}.xml"
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Error fetching UniProt entry for {uniprot_id}")
    
        # Parse the XML response
        root = ET.fromstring(response.content)
        pdb_ids = []
        # Iterate through cross-references to find PDB entries
        for cross_ref in root.findall(".//{http://uniprot.org/uniprot}dbReference"):
            if cross_ref.attrib.get('type') == 'PDB':
                pdb_ids.append(cross_ref.attrib.get('id'))
        
        if not pdb_ids:
            print(f"No PDB IDs found for UniProt ID {uniprot_id}.")
            raise Exception(f"No PDB IDs found for UniProt ID {uniprot_id}.")
    
        pdb_id = pdb_ids[0]
    
        """Fetch the PDB file content from the PDB database."""
        url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
        response = requests.get(url)
        if response.status_code == 200:
            with open(temp_protpdb,"w") as f:
                f.write(response.text)
            os.system(f"obabel {temp_protpdb} -O {temp_protpdbqt} -xh -xr")
            # clean_pdb(temp_protpdb)
            return temp_protpdbqt
        else:
            raise Exception(f"Error fetching PDB file for {pdb_id}")

In [22]:
def run_docking(protein_file, ligand_file, center, box_size):
    """
    Perform molecular docking using AutoDock Vina
    """
    v = Vina()
    
    # Prepare receptor
    v.set_receptor(protein_file)
    
    # Set docking box
    v.compute_vina_maps(center=center, box_size=box_size)
    
    # Dock ligand
    v.set_ligand_from_file(ligand_file)

    v.dock()
    
    # Save results
    v.write_poses("docked_poses.pdbqt", n_poses=1)
    
    return "docked_poses.pdbqt"

In [23]:
def visualize_docking(protein_file, docked_poses_file):
    """
    Create interactive 3D visualization of docking results
    """
    view = py3Dmol.view(width=800, height=600)
    
    # Load protein
    view.addModel(open(protein_file).read(), "pdb")
    view.setStyle({'model': -1}, {'cartoon': {'color': 'spectrum'}})
    
    # Load docked ligand
    view.addModel(open(docked_poses_file).read(), "pdbqt")
    view.setStyle({'model': 1}, {'stick': {'colorscheme': 'greenCarbon'}})
    
    # Set view options
    view.zoomTo()
    view.spin(True)
    
    return view


def dock_and_visualize(smiles, uniprot_id):
    """
    Main function to perform docking and visualization
    """
    # Prepare structures
    ligand_file = prepare_ligand(smiles)
    protein_file = prepare_protein(uniprot_id)
    
    # Define docking box (this should be customized based on your protein)
    center = [0.0, 0.0, 0.0]  # Center coordinates
    box_size = [20, 20, 20]   # Box dimensions
    
    # Run docking
    docked_poses = run_docking(protein_file, ligand_file, center, box_size)
    
    # Create visualization
    view = visualize_docking(protein_file, docked_poses)
    
    return view

In [24]:
# Now let's test with some sample inputs
# Sample SMILES for a drug (Aspirin)
drug_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"  # Aspirin

# Sample UniProt ID for a protein (example: P12345)
protein_uniprot = "Q852Q0"  # Replace with valid UniProt ID

view = dock_and_visualize(drug_smiles,protein_uniprot)
view.show()

1 molecule converted


Checking in AlphaFold database:....


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is temp_prot.pdb)

1 molecule converted


Computing Vina grid ... done.


TypeError: 

PDBQT parsing error: Unknown or inappropriate tag found in flex residue or ligand.
 > ATOM      1  C   UNL     1       1.009  -0.897  -0.600  0.00  0.00    +0.000 C 
